# Analyzing variable importance using logistic regression
### Christian Igel, 2021

This notebook demonstrates how to analyze variable importance using logistic regression. It reimplements the example in R from https://stats.idre.ucla.edu/r/dae/logit-regression/ in Python. It is reassuring to reproduce the R results. 

It is important that we do not use the logistic regression from scikit-learn but from [statmodels](https://www.statsmodels.org/). First, we will do the data preprocessing using [pandas](https://pandas.pydata.org/) to organize the data, then we will make use of [patsy](https://patsy.readthedocs.io).

Any suggestions for improvements are more than welcome.

In [1]:
import pandas as pd
import statsmodels.api as sm

The task is to predict admission into graduate school based on  GRE (Graduate Record Exam scores), GPA (grade point average) and prestige of the undergraduate institution. Let's load the data:

In [2]:
df = pd.read_csv("https://stats.idre.ucla.edu/stat/data/binary.csv")

Let's inspect the data:

In [3]:
df.head(5)

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [4]:
df.describe()

,admit,gre,gpa,rank
count,400.000000,400.000000,400.000000,400.00000
mean,0.317500,587.700000,3.389900,2.48500
std,0.466087,115.516536,0.380567,0.94446
min,0.000000,220.000000,2.260000,1.00000
25%,0.000000,520.000000,3.130000,2.00000
50%,0.000000,580.000000,3.395000,2.00000
75%,1.000000,660.000000,3.670000,3.00000
max,1.000000,800.000000,4.000000,4.00000


#### First version
In the first version, we do the main data preprocessing steps ourselves.

The goal is to predict/explain the binary variable `admit` given the other variables. Thus, we split data into input (predictor variables) and target (response/dependent variable):

In [5]:
X = df.iloc[:,1:4]
y = df.iloc[:,0]

We treat the `rank`, which indicates the prestige of the undergraduate institution, as a categorical variable:

In [6]:
X["rank"] = X["rank"].astype('category')

In [7]:
X

,gre,gpa,rank
0,380,3.61,3
1,660,3.67,3
2,800,4.00,1
3,640,3.19,4
4,520,2.93,4
...,...,...,...
395,620,4.00,2
396,560,3.04,3
397,460,2.63,2
398,700,3.65,2


In [8]:
display(y)

0      0
1      1
2      1
3      1
4      0
      ..
395    0
396    0
397    0
398    0
399    0
Name: admit, Length: 400, dtype: int64

Now we transform the categorical variable. Note that in contrast to a one-hot-encoding one column is dropped to avoid the linear dependency. 

In [9]:
X_transformed = pd.get_dummies(X, prefix=['rank'], drop_first=True)
X_transformed

,gre,gpa,rank_2,rank_3,rank_4
0,380,3.61,0,1,0
1,660,3.67,0,1,0
2,800,4.00,0,0,0
3,640,3.19,0,0,1
4,520,2.93,0,0,1
...,...,...,...,...,...
395,620,4.00,1,0,0
396,560,3.04,0,1,0
397,460,2.63,1,0,0
398,700,3.65,1,0,0


The logistic regression model we are using does not have a built in intercept (bias/offset) parameter. Thus, we augment our input data with a constant dummy variable. 

In [10]:
X_transformed = sm.add_constant(X_transformed)
X_transformed

,const,gre,gpa,rank_2,rank_3,rank_4
0,1.0,380,3.61,0,1,0
1,1.0,660,3.67,0,1,0
2,1.0,800,4.00,0,0,0
3,1.0,640,3.19,0,0,1
4,1.0,520,2.93,0,0,1
...,...,...,...,...,...,...
395,1.0,620,4.00,1,0,0
396,1.0,560,3.04,0,1,0
397,1.0,460,2.63,1,0,0
398,1.0,700,3.65,1,0,0


Now we can compute and inspect the logistic regression model: 

In [11]:
logit_model=sm.Logit(y, X_transformed)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.573147
         Iterations 6
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.083     
Dependent Variable: admit            AIC:              470.5175  
Date:               2021-09-27 11:56 BIC:              494.4663  
No. Observations:   400              Log-Likelihood:   -229.26   
Df Model:           5                LL-Null:          -249.99   
Df Residuals:       394              LLR p-value:      7.5782e-08
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
-------------------------------------------------------------------
           Coef.    Std.Err.      z      P>|z|     [0.025    0.975]
-------------------------------------------------------------------
const     -3.9900     1.1400   -3.5001   0.0005   -6.2242   -1.7557
gre        0.0023     0.0011    2.0699   0.0385    0.0001 

#### Second version

In the second version, we use a library function for creating design matrices:

In [12]:
from patsy import dmatrices

We can use a formula syntax to specify the design matrix as in R. When a formula is used to specify the terms to include in the design martrix, a constant for the intercept term will be included by default. The function `C` handles the encoding of the categorical varibale for us:

In [13]:
y, X = dmatrices('admit ~ gre + gpa + C(rank)', df, return_type = 'dataframe')
X.head()

,Intercept,C(rank)[T.2],C(rank)[T.3],C(rank)[T.4],gre,gpa
0,1.0,0.0,1.0,0.0,380.0,3.61
1,1.0,0.0,1.0,0.0,660.0,3.67
2,1.0,0.0,0.0,0.0,800.0,4.00
3,1.0,0.0,0.0,1.0,640.0,3.19
4,1.0,0.0,0.0,1.0,520.0,2.93


In [14]:
logit = sm.Logit(y, X)
result = logit.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.573147
         Iterations 6
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.083     
Dependent Variable: admit            AIC:              470.5175  
Date:               2021-09-27 11:56 BIC:              494.4663  
No. Observations:   400              Log-Likelihood:   -229.26   
Df Model:           5                LL-Null:          -249.99   
Df Residuals:       394              LLR p-value:      7.5782e-08
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
Intercept     -3.9900    1.1400  -3.5001  0.0005  -6.2242  -1.7557
C(rank)[T.2]  -0.6754    0.3165  -2.1342  0.0328  -1.2958  -0.

#### Interpreting the resuls

The results table  shows the coefficients, their standard errors, the z-statistic (sometimes called a Wald z-statistic, in this case the z-value is computed as the coefficient over its standard error, a concept we have not discussed in the lecture), the associated p-values (P), as well as the confidence intervals for the coefficient estimates. 

Assuming a 5% significance level, one can argue that all predictor variables are statistically significant, because their p-values are all smaller than 0.05. (The null hypothesis is that the coefficient is zero. If the p-value is smaller than our significnace level, we reject the null hypothesis.) 

"The logistic regression coefficients give the change in the log odds of the outcome for a one unit increase in the predictor variable. For every one unit change in `gre`, the log odds of admission (versus non-admission) increases by 0.002. For a one unit increase in `gpa`, the log odds of being admitted to graduate school increases by 0.804. The indicator variables for `rank` have a slightly different interpretation. For example, having attended an undergraduate institution with `rank` of 2, versus an institution with a `rank` of 1, changes the log odds of admission by -0.675" [(quoted from here)](https://stats.idre.ucla.edu/r/dae/logit-regression/).

The accuracy of the model on the training set, which is *not* an ubiased estimate of the generalizaition performance, can be computed as follows:

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = result.predict(X)  # Predict using the result object
print('Accuracy on training set:', 
      accuracy_score(y, [1 if m >= 0.5 else 0 for m in y_pred]))

#### Additional comments
In the following, it is demonstrated that linear rescaling does not change the significance of the the predictor variables.

In [ ]:
from sklearn.preprocessing import StandardScaler
cols_to_norm = ['gre','gpa']
X[cols_to_norm] = StandardScaler().fit_transform(X[cols_to_norm])

In [ ]:
X.head()

In [ ]:
logit = sm.Logit(y, X)
result = logit.fit()
print(result.summary2())

Note that the p-values of the predictor variables did not change, as it should be. 
(However, we can not reject the null hypothesis that the intercept parameter is zero, zero is also in the confidence interval of the intercept estimate.) 